In [1]:
import numpy as np
import casadi as ca
import scipy.io
import matplotlib.pyplot as plt
from dynamics import get_linear_dynamics

In [2]:
# M players
M = 1

# N horizon length
N = 10

# n state size
n = 4  # (x, y, v_x, v_y)

# m control input size
m = 2  # (a_x, a_y)

# simulation length
sim_length = 10

# timestep
dt = 0.1 

# simulation steps
L = int(sim_length/dt)

# collision avoidance radius
r = 0.5

# running cost matrix
Q = np.eye(n)

# terminal cost matrix
Qf = np.eye(n)

# control cost matrix
R = np.eye(m)

In [30]:
def generate(x0, M, N, n, m, dt):
    # x0: list of (n,) numpy arrays
    # M: num players
    # N: horizon
    # n: state size
    # m: control input size

    # integrate separately for each player, assuming
    # zero input as initial guess and all players have
    # identical dynamics
    X = np.empty([])
    A = get_linear_dynamics( n, m, dt)[0]
    for i in range(M):
        x = x0[i]
        for j in range(N):
            x = A@x
            X = np.hstack((X, x))
            
    U = np.zeros((M*N*m,)) # guess zero input
    mu = np.ones((M*N*n,)) # guess multipliers of 1

    y0 = np.hstack((X,U,mu))
    return y0, X, U, mu

In [31]:
[np.zeros((4,))]*4

[array([0., 0., 0., 0.]),
 array([0., 0., 0., 0.]),
 array([0., 0., 0., 0.]),
 array([0., 0., 0., 0.])]

In [32]:
list_x0 = [np.zeros((4,))]*4
print(list_x0[0])

[0. 0. 0. 0.]


In [33]:
y0, X_guess, U_guess, mu_guess = generate(list_x0, M, N, n, m, dt)  # [X, U, mu]

In [ ]:
# System setup
A = np.array([[1, 0],
              [0, 1.1]])
B = np.array([[1],
              [1]])

nx = A.shape[0]
nu = B.shape[1]

# Cost matrices
Q1 = np.eye(nx)
Q2 = 100 * np.array([[3, -2], [-2, 4]])
R = np.eye(nu)
Q = Q1
P = Q

# Horizon
N = 12

In [ ]:
# Define CasADi variables
x0 = ca.MX.sym('x0', nx)
U = ca.MX.sym('U', nu, N)

# Build optimization
x = x0
cost = 0
g = []  # constraints

for k in range(N):
    u_k = U[:, k]
    cost += ca.mtimes([x.T, Q, x]) + ca.mtimes([u_k.T, R, u_k])
    x = ca.mtimes(A, x) + ca.mtimes(B, u_k)
    g.append(u_k - 1)   # u_k <= 1
    g.append(-u_k - 1)  # u_k >= -1
    g.append(x - np.array([[5], [5]]))  # x <= 5
    g.append(-x - np.array([[5], [5]])) # x >= -5

# Terminal cost
cost += ca.mtimes([x.T, P, x])

# Flatten constraints
g_flat = ca.vertcat(*g)

# Define optimization problem
opt_vars = ca.vec(U)
qpsol = {'x': opt_vars, 'f': cost, 'g': g_flat, 'p': x0}
# opts = {'qpoases.print_level':0, 'print_time':0}
solver = ca.qpsol('S', 'qpoases', qpsol)

# Bounds
nlb = []
nub = []

for k in range(N):
    # u constraints
    nlb += [-np.inf]*nu + [-np.inf]*nu
    nub += [0]*nu + [0]*nu

    # x constraints
    nlb += [-np.inf]*nx + [-np.inf]*nx
    nub += [0]*nx + [0]*nx


In [ ]:

# Simulation
x_ic = np.array([[1.0], [1.0]])
X = [x_ic.copy()]
U_applied = []

for i in range(200):
    # print(f"Iteration {i+1}")
    sol = solver(x0=ca.DM.zeros(nu*N, 1),
                 p=x_ic,
                 lbg=nlb,
                 ubg=nub)

    u_star = sol['x'].full().reshape(nu, N)
    u_apply = u_star[:, 0:1]
    x_ic = A @ x_ic + B @ u_apply
    X.append(x_ic.copy())
    U_applied.append(u_apply.item())

# Plot trajectory
X = np.hstack(X)
plt.plot(X[0, :], X[1, :], '*-b')
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('MPC trajectory using CasADi')
plt.grid()
plt.show()